In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jaguar-re-id/sample_submission.csv
/kaggle/input/jaguar-re-id/train.csv
/kaggle/input/jaguar-re-id/test.csv
/kaggle/input/jaguar-re-id/test/test/test_0021.png
/kaggle/input/jaguar-re-id/test/test/test_0132.png
/kaggle/input/jaguar-re-id/test/test/test_0201.png
/kaggle/input/jaguar-re-id/test/test/test_0209.png
/kaggle/input/jaguar-re-id/test/test/test_0112.png
/kaggle/input/jaguar-re-id/test/test/test_0207.png
/kaggle/input/jaguar-re-id/test/test/test_0024.png
/kaggle/input/jaguar-re-id/test/test/test_0330.png
/kaggle/input/jaguar-re-id/test/test/test_0278.png
/kaggle/input/jaguar-re-id/test/test/test_0215.png
/kaggle/input/jaguar-re-id/test/test/test_0113.png
/kaggle/input/jaguar-re-id/test/test/test_0124.png
/kaggle/input/jaguar-re-id/test/test/test_0018.png
/kaggle/input/jaguar-re-id/test/test/test_0063.png
/kaggle/input/jaguar-re-id/test/test/test_0359.png
/kaggle/input/jaguar-re-id/test/test/test_0314.png
/kaggle/input/jaguar-re-id/test/test/test_0244.png
/kaggle/inp

In [3]:
import os
import math
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

import cv2
import timm

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [4]:
# -------------------------
# Config
# -------------------------
@dataclass
class CFG:
    seed: int = 42
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    # Paths
    DATA_ROOT: str = "/path/to/DATA_ROOT"
    TRAIN_CSV: str = "train.csv"
    TEST_CSV: str = "test.csv"
    SAMPLE_SUB: str = "sample_submission.csv"
    TRAIN_IMG_DIR: str = "train_images"
    TEST_IMG_DIR: str = "test_images"

    # Image / aug
    img_size: int = 256
    use_alpha: bool = True  # if your PNGs have alpha cutouts

    # Model
    backbone: str = "convnext_base"  # also good: "vit_base_patch16_384"
    embed_dim: int = 512

    # ArcFace
    arc_s: float = 30.0
    arc_m: float = 0.30

    # Training
    epochs: int = 12
    lr: float = 3e-4
    weight_decay: float = 1e-4
    warmup_epochs: int = 1

    # PK sampling
    P: int = 16
    K: int = 4  # batch = P*K

    # Split for local eval (within-identity holdout)
    val_frac: float = 0.2


cfg = CFG()

In [5]:
cfg.DATA_ROOT = "/kaggle/input/jaguar-re-id"

cfg.TRAIN_IMG_DIR = "train/train"
cfg.TEST_IMG_DIR  = "test/test"

In [6]:
def set_seed(seed: int):
    print("[DEBUG] Setting random seeds")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [7]:
def read_image(path: str, use_alpha: bool = True) -> np.ndarray:
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"[ERROR] Image not found: {path}")

    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    if img.shape[2] == 4 and use_alpha:
        bgr = img[:, :, :3].astype(np.float32)
        alpha = img[:, :, 3:4].astype(np.float32) / 255.0
        bg = np.random.randint(0, 256, size=(1, 1, 3)).astype(np.float32)
        img = (bgr * alpha + bg * (1 - alpha)).astype(np.uint8)
    else:
        img = img[:, :, :3]

    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [8]:
# =====================================================
# TRANSFORMS
# =====================================================
class TrainTransform:
    def __init__(self, size):
        self.size = size

    def __call__(self, img):
        img = cv2.resize(img, (self.size, self.size))
        if random.random() < 0.5:
            img = img[:, ::-1].copy()
        x = torch.from_numpy(img).float().permute(2, 0, 1) / 255.0
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        return (x - mean) / std

In [9]:
class ValTransform(TrainTransform):
    pass

In [10]:
# =====================================================
# DATASETS
# =====================================================
class JaguarTrainDataset(Dataset):
    def __init__(self, df, img_dir, transform, label2idx):
        print(f"[DEBUG] Initializing TrainDataset with {len(df)} samples")
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.label2idx = label2idx

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row["filename"])
        img = read_image(path)
        x = self.transform(img)
        y = self.label2idx[row["ground_truth"]]
        return x, y


class JaguarImageDataset(Dataset):
    def __init__(self, filenames, img_dir, transform):
        print(f"[DEBUG] Initializing ImageDataset with {len(filenames)} images")
        self.filenames = filenames
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        fn = self.filenames[idx]
        path = os.path.join(self.img_dir, fn)
        img = read_image(path)
        x = self.transform(img)
        return x, fn

In [11]:
# =====================================================
# PK SAMPLER
# =====================================================
class PKSampler(Sampler):
    def __init__(self, labels, P, K):
        print("[DEBUG] Initializing PKSampler")
        self.labels = np.array(labels)
        self.P = P
        self.K = K

        self.label_to_indices = {}
        for i, y in enumerate(self.labels):
            self.label_to_indices.setdefault(int(y), []).append(i)

        self.unique_labels = list(self.label_to_indices.keys())
        self.batches = len(self.labels) // (P * K)

        print(f"[DEBUG] PKSampler batches per epoch: {self.batches}")

    def __iter__(self):
        for _ in range(self.batches):
            selected = random.sample(self.unique_labels, self.P)
            batch = []
            for y in selected:
                inds = self.label_to_indices[y]
                batch.extend(random.choices(inds, k=self.K))
            yield batch

    def __len__(self):
        return self.batches

In [12]:
from tqdm import tqdm
import time

In [13]:
class ArcMarginProduct(nn.Module):
    """
    ArcFace head with dynamic margin scheduling support.
    """
    def __init__(self, in_features, out_features, s=30.0, m=0.30, easy_margin=False):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.easy_margin = easy_margin

        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self._update_margin_constants()

    def _update_margin_constants(self):
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.th = math.cos(math.pi - self.m)
        self.mm = math.sin(math.pi - self.m) * self.m

    def set_margin(self, new_m: float):
        self.m = float(new_m)
        self._update_margin_constants()
        print(f"[DEBUG] ArcFace margin set to m={self.m:.4f}")

    def forward(self, embeddings, labels):
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.weight))
        sine = torch.sqrt(torch.clamp(1.0 - cosine * cosine, min=0.0))
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1), 1.0)

        logits = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        logits *= self.s
        return logits

In [14]:
def arcface_margin_schedule(epoch, total_epochs, m_start=0.10, m_end=0.35):
    """
    Linear ArcFace margin schedule.
    """
    if total_epochs <= 1:
        return m_end

    t = (epoch - 1) / (total_epochs - 1)
    return float(m_start + t * (m_end - m_start))

In [15]:
class ReIDModel(nn.Module):
    def __init__(self, backbone_name, embed_dim, num_classes, arc_s, arc_m):
        super().__init__()
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=True,
            num_classes=0,
            global_pool="avg"
        )
        feat_dim = self.backbone.num_features

        self.embedding = nn.Sequential(
            nn.Linear(feat_dim, embed_dim),
            nn.BatchNorm1d(embed_dim)
        )

        self.arc = ArcMarginProduct(
            embed_dim,
            num_classes,
            s=arc_s,
            m=arc_m
        )

    def forward(self, x, labels=None):
        feats = self.backbone(x)
        emb = self.embedding(feats)
        emb = F.normalize(emb, dim=1)

        if labels is None:
            return emb

        logits = self.arc(emb, labels)
        return logits, emb

In [16]:
from tqdm import tqdm
import time

def train_one_epoch(model, loader, optimizer, device, epoch):
    model.train()
    ce = nn.CrossEntropyLoss()

    total_loss = 0.0
    start_time = time.time()

    pbar = tqdm(
        loader,
        desc=f"Epoch {epoch}",
        leave=True
    )

    for step, (xb, yb) in enumerate(pbar):
        step_start = time.time()

        xb = xb.to(device)
        yb = yb.to(device)

        logits, _ = model(xb, yb)
        loss = ce(logits, yb)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)

        step_time = time.time() - step_start

        # live progress bar update
        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "step_s": f"{step_time:.1f}"
        })

    epoch_time = (time.time() - start_time) / 60.0
    avg_loss = total_loss / len(loader.dataset)

    print(
        f"[DEBUG] Epoch {epoch} finished | "
        f"avg_loss={avg_loss:.6f} | "
        f"time={epoch_time:.2f} min"
    )

    return avg_loss


In [17]:
def stratified_within_id_split(df, val_frac=0.2, seed=42):
    """
    Splits the dataset into train/val by holding out a fraction of images
    WITHIN each identity (ground_truth).

    This is the correct split strategy for re-identification.
    """
    rng = np.random.RandomState(seed)

    train_parts = []
    val_parts = []

    for identity, group in df.groupby("ground_truth"):
        indices = group.index.to_numpy()
        rng.shuffle(indices)

        if len(indices) == 1:
            # Only one image → keep it in train
            train_parts.append(df.loc[indices])
            continue

        n_val = max(1, int(len(indices) * val_frac))
        val_idx = indices[:n_val]
        train_idx = indices[n_val:]

        train_parts.append(df.loc[train_idx])
        val_parts.append(df.loc[val_idx])

    train_df = (
        pd.concat(train_parts)
        .sample(frac=1.0, random_state=seed)
        .reset_index(drop=True)
    )

    val_df = (
        pd.concat(val_parts)
        .sample(frac=1.0, random_state=seed)
        .reset_index(drop=True)
    )

    print(
        f"[DEBUG] Stratified split complete | "
        f"train={len(train_df)} val={len(val_df)}"
    )

    return train_df, val_df


In [18]:
import os
import json
import time
import math
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from tqdm import tqdm


def main():
    
    set_seed(cfg.seed)

    print("\n========== STARTING PIPELINE ==========")

    # --------------------------------------------------
    # Prepare artifact directory FIRST (critical)
    # --------------------------------------------------
    ARTIFACT_DIR = "/kaggle/working/artifacts"
    os.makedirs(ARTIFACT_DIR, exist_ok=True)
    print(f"[DEBUG] Artifacts will be saved to: {ARTIFACT_DIR}")

    # --------------------------------------------------
    # Load CSVs
    # --------------------------------------------------
    train_csv = os.path.join(cfg.DATA_ROOT, cfg.TRAIN_CSV)
    test_csv  = os.path.join(cfg.DATA_ROOT, cfg.TEST_CSV)

    df_train = pd.read_csv(train_csv)
    df_test  = pd.read_csv(test_csv)

    print(f"[DEBUG] Train samples: {len(df_train)}")
    print(f"[DEBUG] Test pairs: {len(df_test)}")

    # --------------------------------------------------
    # Label mapping
    # --------------------------------------------------
    labels = sorted(df_train["ground_truth"].unique())
    label2idx = {l: i for i, l in enumerate(labels)}
    df_train["label"] = df_train["ground_truth"].map(label2idx)

    num_classes = len(labels)
    print(f"[DEBUG] Num identities: {num_classes}")

    # --------------------------------------------------
    # Paths
    # --------------------------------------------------
    train_img_dir = os.path.join(cfg.DATA_ROOT, cfg.TRAIN_IMG_DIR)

    # --------------------------------------------------
    # Stratified split
    # --------------------------------------------------
    tr_df, va_df = stratified_within_id_split(
        df_train, cfg.val_frac, cfg.seed
    )

    print(f"[DEBUG] Train split: {len(tr_df)} | Val split: {len(va_df)}")

    # --------------------------------------------------
    # Datasets & loaders
    # --------------------------------------------------
    tr_ds = JaguarTrainDataset(
        tr_df,
        train_img_dir,
        TrainTransform(cfg.img_size),
        label2idx
    )

    tr_sampler = PKSampler(
        tr_df["label"].tolist(),
        P=cfg.P,
        K=cfg.K
    )

    tr_loader = DataLoader(
        tr_ds,
        batch_sampler=tr_sampler,
        num_workers=2,
        pin_memory=True
    )

    # --------------------------------------------------
    # Model & optimizer
    # --------------------------------------------------
    model = ReIDModel(
        cfg.backbone,
        cfg.embed_dim,
        num_classes,
        cfg.arc_s,
        cfg.arc_m
    ).to(cfg.device)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=cfg.lr,
        weight_decay=cfg.weight_decay
    )

    history = []

    # --------------------------------------------------
    # Training loop
    # --------------------------------------------------
    for epoch in range(1, cfg.epochs + 1):

        print(f"\n[DEBUG] Epoch {epoch}/{cfg.epochs}")

        # ArcFace margin scheduling
        new_m = arcface_margin_schedule(
            epoch,
            cfg.epochs,
            m_start=0.10,
            m_end=cfg.arc_m
        )
        model.arc.set_margin(new_m)

        # Train
        loss = train_one_epoch(
            model,
            tr_loader,
            optimizer,
            cfg.device,
            epoch
        )

        history.append({
            "epoch": int(epoch),
            "loss": float(loss),
            "arc_margin": float(new_m)
        })

    # --------------------------------------------------
    # Save artifacts (SAFE ZONE)
    # --------------------------------------------------
    print("\n========== SAVING ARTIFACTS ==========")

    torch.save(
        model.state_dict(),
        os.path.join(ARTIFACT_DIR, "final_model.pt")
    )
    print("[DEBUG] Model saved")

    with open(os.path.join(ARTIFACT_DIR, "train_history.json"), "w") as f:
        json.dump(history, f, indent=2)
    print("[DEBUG] train_history.json saved")

    with open(os.path.join(ARTIFACT_DIR, "config.json"), "w") as f:
        json.dump(cfg.__dict__, f, indent=2)
    print("[DEBUG] config.json saved")

    print("\n========== TRAINING COMPLETE ==========")
    print("[INFO] All artifacts saved successfully")

    return model, history

In [19]:
if __name__ == "__main__":
    model, history = main()

[DEBUG] Setting random seeds

========== STARTING PIPELINE ==========
[DEBUG] Artifacts will be saved to: /kaggle/working/artifacts
[DEBUG] Train samples: 1895
[DEBUG] Test pairs: 137270
[DEBUG] Num identities: 31
[DEBUG] Stratified split complete | train=1528 val=367
[DEBUG] Train split: 1528 | Val split: 367
[DEBUG] Initializing TrainDataset with 1528 samples
[DEBUG] Initializing PKSampler
[DEBUG] PKSampler batches per epoch: 23


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]


[DEBUG] Epoch 1/12
[DEBUG] ArcFace margin set to m=0.1000


Epoch 1:   0%|          | 0/23 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1: 100%|██████████| 23/23 [46:30<00:00, 121.32s/it, loss=1.3653, step_s=108.5] 


[DEBUG] Epoch 1 finished | avg_loss=2.871213 | time=46.51 min

[DEBUG] Epoch 2/12
[DEBUG] ArcFace margin set to m=0.1182


...

[DEBUG] Epoch 10 finished | avg_loss=0.030969 | time=47.65 min

[DEBUG] Epoch 11/12
[DEBUG] ArcFace margin set to m=0.2818


Epoch 11: 100%|██████████| 23/23 [47:28<00:00, 123.84s/it, loss=0.0186, step_s=112.7] 


[DEBUG] Epoch 11 finished | avg_loss=0.028990 | time=47.47 min

[DEBUG] Epoch 12/12
[DEBUG] ArcFace margin set to m=0.3000


Epoch 12: 100%|██████████| 23/23 [47:02<00:00, 122.71s/it, loss=0.0049, step_s=97.2]  


[DEBUG] Epoch 12 finished | avg_loss=0.035982 | time=47.04 min

========== SAVING ARTIFACTS ==========
[DEBUG] Model saved
[DEBUG] train_history.json saved
[DEBUG] config.json saved

========== TRAINING COMPLETE ==========
[INFO] All artifacts saved successfully


In [20]:
import json
import torch
import numpy as np
import pandas as pd

ARTIFACT_DIR = "/kaggle/working/artifacts"

print("[DEBUG] Loading artifacts from:", ARTIFACT_DIR)

with open(f"{ARTIFACT_DIR}/config.json", "r") as f:
    saved_cfg = json.load(f)

with open(f"{ARTIFACT_DIR}/train_history.json", "r") as f:
    train_history = json.load(f)

print("[DEBUG] Loaded config keys:", list(saved_cfg.keys()))
print("[DEBUG] Loaded training epochs:", len(train_history))


[DEBUG] Loading artifacts from: /kaggle/working/artifacts
[DEBUG] Loaded config keys: ['seed', 'device', 'DATA_ROOT', 'TRAIN_CSV', 'TEST_CSV', 'SAMPLE_SUB', 'TRAIN_IMG_DIR', 'TEST_IMG_DIR', 'img_size', 'use_alpha', 'backbone', 'embed_dim', 'arc_s', 'arc_m', 'epochs', 'lr', 'weight_decay', 'warmup_epochs', 'P', 'K', 'val_frac']
[DEBUG] Loaded training epochs: 12


In [21]:
import pandas as pd

hist = pd.DataFrame(train_history)

print("\n========== TRAINING DYNAMICS ==========")
print(hist)

print("\n[SUMMARY]")
print("Initial loss:", hist.loss.iloc[0])
print("Final loss  :", hist.loss.iloc[-1])
print("Min loss    :", hist.loss.min())
print("Max margin  :", hist.arc_margin.max())

print("\n[EXPECTED]")
print("- Loss should decrease more gradually than baseline")
print("- Loss should NOT collapse to near-zero too early")
print("- ArcFace margin should increase every epoch")



========== TRAINING DYNAMICS ==========
    epoch      loss  arc_margin
0       1  2.871213    0.100000
1       2  0.530760    0.118182
2       3  0.197410    0.136364
3       4  0.116373    0.154545
4       5  0.082355    0.172727
5       6  0.061441    0.190909
6       7  0.083310    0.209091
7       8  0.049333    0.227273
8       9  0.032600    0.245455
9      10  0.030969    0.263636
10     11  0.028990    0.281818
11     12  0.035982    0.300000

[SUMMARY]
Initial loss: 2.871213323782876
Final loss  : 0.035982157830246456
Min loss    : 0.028990154198483022
Max margin  : 0.3

[EXPECTED]
- Loss should decrease more gradually than baseline
- Loss should NOT collapse to near-zero too early
- ArcFace margin should increase every epoch


In [22]:
from torch.utils.data import DataLoader

# rebuild label mapping
df_train = pd.read_csv(
    os.path.join(cfg.DATA_ROOT, cfg.TRAIN_CSV)
)
labels = sorted(df_train["ground_truth"].unique())
label2idx = {l: i for i, l in enumerate(labels)}

num_classes = len(labels)

model = ReIDModel(
    cfg.backbone,
    cfg.embed_dim,
    num_classes,
    cfg.arc_s,
    cfg.arc_m
).to(cfg.device)

model.load_state_dict(
    torch.load(f"{ARTIFACT_DIR}/final_model.pt", map_location=cfg.device)
)

model.eval()
print("[DEBUG] Model loaded and ready for evaluation")


[DEBUG] Model loaded and ready for evaluation


In [23]:
@torch.no_grad()
def extract_embeddings(model, loader, device):
    model.eval()
    emb_dict = {}

    for xb, fns in tqdm(loader, desc="Extracting embeddings"):
        xb = xb.to(device)
        emb = model(xb)          # <-- your model already returns embeddings
        emb = emb.cpu().numpy()

        for fn, e in zip(fns, emb):
            emb_dict[fn] = e

    return emb_dict


In [24]:
@torch.no_grad()
def extract_embeddings_with_labels(model, loader, device):
    model.eval()
    all_emb = []
    all_lbl = []

    for xb, yb in loader:
        xb = xb.to(device)
        emb = model(xb)  # embedding-only forward
        all_emb.append(emb.cpu().numpy())
        all_lbl.append(yb.numpy())

    return (
        np.concatenate(all_emb, axis=0),
        np.concatenate(all_lbl, axis=0)
    )

In [25]:
def identity_balanced_map(embeddings, labels):
    """
    Macro-averaged mAP across identities.
    """
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    sims = embeddings @ embeddings.T

    id_to_indices = {}
    for i, y in enumerate(labels):
        id_to_indices.setdefault(int(y), []).append(i)

    ap_per_id = []

    for y, idxs in id_to_indices.items():
        ap_list = []
        for qi in idxs:
            order = np.argsort(-sims[qi])
            order = order[order != qi]

            rel = (labels[order] == y).astype(int)
            if rel.sum() == 0:
                ap_list.append(0.0)
                continue

            hits = 0
            precisions = []
            for k, r in enumerate(rel, start=1):
                if r:
                    hits += 1
                    precisions.append(hits / k)
            ap_list.append(np.mean(precisions))

        ap_per_id.append(np.mean(ap_list))

    return float(np.mean(ap_per_id))


In [26]:
# rebuild validation loader (same split logic!)
tr_df, va_df = stratified_within_id_split(
    df_train, cfg.val_frac, cfg.seed
)

val_ds = JaguarTrainDataset(
    va_df,
    os.path.join(cfg.DATA_ROOT, cfg.TRAIN_IMG_DIR),
    ValTransform(cfg.img_size),
    label2idx
)

val_loader = DataLoader(
    val_ds,
    batch_size=64,
    shuffle=False,
    num_workers=2
)

emb, lbl = extract_embeddings_with_labels(
    model, val_loader, cfg.device
)

val_map = identity_balanced_map(emb, lbl)

print("\n========== VALIDATION RESULTS ==========")
print("Validation identity-balanced mAP:", round(val_map, 5))

print("\n[EMBEDDING HEALTH]")
norms = np.linalg.norm(emb, axis=1)
print("Embedding norm mean/std:", norms.mean(), norms.std())
print("Embedding norm min/max :", norms.min(), norms.max())

print("\n[EXPECTED]")
print("- mAP should be > baseline")
print("- Norm mean ≈ 1.0")
print("- Norm std small (<0.05)")


[DEBUG] Stratified split complete | train=1528 val=367
[DEBUG] Initializing TrainDataset with 367 samples

========== VALIDATION RESULTS ==========
Validation identity-balanced mAP: 0.81676

[EMBEDDING HEALTH]
Embedding norm mean/std: 1.0 4.422041e-08
Embedding norm min/max : 0.9999999 1.0000001

[EXPECTED]
- mAP should be > baseline
- Norm mean ≈ 1.0
- Norm std small (<0.05)


In [27]:
summary = {
    "validation_identity_balanced_mAP": float(val_map),
    "embedding_norm_mean": float(norms.mean()),
    "embedding_norm_std": float(norms.std()),
    "epochs": len(train_history),
    "final_arc_margin": train_history[-1]["arc_margin"],
}

with open(f"{ARTIFACT_DIR}/evaluation_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("[INFO] Evaluation summary saved to artifacts/")

[INFO] Evaluation summary saved to artifacts/


In [28]:
import json

with open('/kaggle/working/artifacts/evaluation_summary.json') as f:
    metrics = json.load(f)

metrics

{'validation_identity_balanced_mAP': 0.8167605274212113,
 'embedding_norm_mean': 1.0,
 'embedding_norm_std': 4.422041044449543e-08,
 'epochs': 12,
 'final_arc_margin': 0.3}

In [30]:
print("\n========== BUILDING SUBMISSION ==========")

test_csv = os.path.join(cfg.DATA_ROOT, cfg.TEST_CSV)
test_img_dir = os.path.join(cfg.DATA_ROOT, cfg.TEST_IMG_DIR)

df_test = pd.read_csv(test_csv)

# Collect all unique test images used in pairs
unique_imgs = pd.unique(
    df_test[["query_image", "gallery_image"]].values.ravel()
)

print(f"[DEBUG] Unique test images: {len(unique_imgs)}")

# Use YOUR existing dataset
test_ds = JaguarImageDataset(
    unique_imgs,
    test_img_dir,
    ValTransform(cfg.img_size)
)

test_loader = DataLoader(
    test_ds,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# Extract embeddings
emb_dict = extract_embeddings(model, test_loader, cfg.device)

# Compute similarities
sims = []
missing = 0

for row in df_test.itertuples(index=False):
    qe = emb_dict.get(row.query_image)
    ge = emb_dict.get(row.gallery_image)

    if qe is None or ge is None:
        sims.append(np.nan)
        missing += 1
    else:
        cos_sim = float(np.dot(qe, ge))          # [-1, 1]
        sim_01 = (cos_sim + 1.0) / 2.0           # [0, 1]
        sims.append(sim_01)


print(f"[DEBUG] Missing pairs: {missing}")

# Write submission
submission = pd.DataFrame({
    "row_id": df_test["row_id"].values,
    "similarity": np.array(sims, dtype=np.float32)
})

SUB_PATH = "/kaggle/working/submission.csv"
submission.to_csv(SUB_PATH, index=False)

print(f"[INFO] submission.csv written to {SUB_PATH}")
print(submission.head())



========== BUILDING SUBMISSION ==========
[DEBUG] Unique test images: 371
[DEBUG] Initializing ImageDataset with 371 images


Extracting embeddings:   0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Extracting embeddings: 100%|██████████| 6/6 [05:03<00:00, 50.64s/it] 


[DEBUG] Missing pairs: 0
[INFO] submission.csv written to /kaggle/working/submission.csv
   row_id  similarity
0       0    0.816951
1       1    0.644937
2       2    0.504619
3       3    0.653313
4       4    0.678273
